Generate evaluation triplets from studies 


1. parse document 
2. extract claims that are accosiated with refereces via LLM and save in json with referenced paper as name
3. generate topic of claim like (sustanability, business, money, or something like that)
4. repeat step 1-3 for multiple reports from diverse topics that care be releated tho like (genai, sustability, business report)
5. bundle 2-3 claims together from multiple related documents into one claim via LLM
6. generate question for bundled claim 
7. store question, claim and added references in json objects
8. repeat step 6 and 7




In [2]:

from langchain_core.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
from pydantic import BaseModel, Field
#from langchain.schema import Document
import os
from typing import List
import tiktoken
from scidownl import scihub_download
import json

from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client

proxy_client = get_proxy_client('gen-ai-hub')
llm = ChatOpenAI(proxy_model_name='gpt-4o', proxy_client=proxy_client)

INPUT_DIR = "reports/"
OUTPUT_DIR = "cited_papers_2/"
DATASET = "evaluation_references.json"


## Load Documents

In [3]:
documents = []
for file in os.listdir(INPUT_DIR):
    if file.endswith(".md"):
        file_path = os.path.join(INPUT_DIR, file)
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()
            documents.append(Document(page_content=content, metadata={"source": file}))
combined_documents_content = "\n\n".join([doc.page_content for doc in documents])

In [4]:
encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(combined_documents_content)
print(len(tokens))

76911


## Extract claims with references using an LLM chain

In [ ]:
class Response(BaseModel):
    claim: str = Field(description="Claim"),
    references: List[str] = Field(description="List of references")
    dois: List[str] = Field(description="List of DOIs corresponding to each reference in the same order as references, or empty if not found")

class Responses(BaseModel):
    responses: list[Response] = Field(description="List of responses")

parser = PydanticOutputParser(pydantic_object=Responses)

extract_prompt = PromptTemplate(
    input_variables=["text"],
    partial_variables={"format_output": parser.get_format_instructions()},
    template="""You are a knowledgeable assistant specializing in academic research analysis. Your task is to identify and extract **research-based claims** from the provided document. Additionally, you will match each inline citation in the claim to its corresponding entry in the references section and extract the DOI from that entry.

**Definition of a Research-Based Claim:**
- A statement that presents a finding, result, or implication derived from empirical studies.
- Typically includes analysis, comparisons, or conclusions drawn from data.
- Supported by one or more inline citations in the format "(Author et al., YEAR), (Author and Author, YEAR) or (Author, YEAR)".
- **The extracted claim should exclude any reference information or inline citations, presenting a standalone statement.**

**Exclude the following from extraction:**
- Definitions or historical accounts.
- General statements without empirical support.
- Descriptions of concepts without presenting findings.

**Tasks:**
1. **Extract each research-based claim without any reference information or inline citations.** The claim should be a standalone statement free of author names, years, or citation markers.
2. For each claim, extract the list of inline citations that appear in that claim.
3. For each inline citation (Author et al., YEAR), match it to the corresponding full reference in the references section (if available within the provided text).
4. From the matched full reference, extract the DOI (if present).
5. If no DOI is found for a given citation, return an empty string for that DOI.

<document>
{text}
</document>

**The output MUST strictly adhere to the following JSON format and include only this JSON without any additional text:**
{format_output}
""", 
)

extract_chain = extract_prompt | llm | parser

claims = []
for doc in documents:
    response = extract_chain.invoke({"text": doc.page_content})
    for res in response.responses:
        claim_dict = res.dict()
        claim_dict["doc_source"] = doc.metadata["source"]
        claims.append(claim_dict)

count = sum(
    1 
    for entry in claims 
    if any(doi.strip() for doi in entry.get('dois', []) if doi)
)
print("Number of claims: ", len(claims))
print("Number of claims with at least one non-empty DOI:", count)


e:\Repositories\graphrag-businessqa-evaluation\.conda\Lib\site-packages\pydantic\json_schema.py:2191: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='Claim'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


Number of claims with at least one non-empty DOI: 84


In [19]:
for claim in claims:    
    print(json.dumps(claim, indent=4, ensure_ascii=False))

{
    "claim": "The accounting sector of a company can promote environmental conservation through environmental costs, and at the same time, improve performance when implementing a Corporate Sustainability Management System (CSMS).",
    "references": [
        "Endiana et al., 2020"
    ],
    "dois": [
        "10.13106/jafeb.2020.vol7.no12.731"
    ],
    "doc_source": "s10668-023-02933-7.md"
}
{
    "claim": "Allocating appropriate environmental costs through CSMS can effectively improve the company’s financial performance.",
    "references": [
        "Endiana et al., 2020"
    ],
    "dois": [
        "10.13106/jafeb.2020.vol7.no12.731"
    ],
    "doc_source": "s10668-023-02933-7.md"
}
{
    "claim": "A proper application of CSMS, with the disclosure of environmental activities and costs, can enhance customer loyalty.",
    "references": [
        "Endiana et al., 2020"
    ],
    "dois": [
        "10.13106/jafeb.2020.vol7.no12.731"
    ],
    "doc_source": "s10668-023-02933-7

In [37]:

filtered_claims = []
unique_dois = set()

# Filter claims and collect DOIs
for claim in claims:
    valid_dois = [doi for doi in claim["dois"] if doi]
    if valid_dois:
        filtered_claims.append(claim)
        unique_dois.update(valid_dois)

# Save unique DOIs to the file
with open("unique_dois.txt", "w", encoding="utf-8") as f:
    for doi in sorted(unique_dois):
        f.write(doi + "\n")

print("Total extracted claims: ", len(claims))
print("Filtered claims with at least one DOI: ", len(filtered_claims))
print("Total unique DOIs saved: ", len(unique_dois))


Total extracted claims:  92
Filtered claims with at least one DOI:  84
Total unique DOIs saved:  66


## Fetch and filter cited Reports 

In [23]:
!python -m PyPaperBot --doi-file=unique_dois.txt --dwn-dir=cited_papers_3 

PyPaperBot v1.4.1
PyPaperBot is a Python tool for downloading scientific papers using Google Scholar, Crossref and SciHub.
        -Join the telegram channel to stay updated --> https://t.me/pypaperbotdatawizards <--
        -If you like this project, you can share a cup of coffee at --> https://www.paypal.com/paypalme/ferru97 <-- :)


Searching paper 1 of 66 with DOI 10.1002/bse.2172
Paper not found 10.1002/bse.2172
Searching paper 2 of 66 with DOI 10.1002/csr.1480
Python 3
Searching paper 3 of 66 with DOI 10.1002/csr.1906
Python 3
Searching paper 4 of 66 with DOI 10.1007/s00187-017-0258-5
Paper not found 10.1007/s00187-017-0258-5
Searching paper 5 of 66 with DOI 10.1007/s10490-014-9391-0
Paper not found 10.1007/s10490-014-9391-0
Searching paper 6 of 66 with DOI 10.1007/s10551-012-1291-2
Paper not found 10.1007/s10551-012-1291-2
Searching paper 7 of 66 with DOI 10.1007/s10551-015-2631-7
Paper not found 10.1007/s10551-015-2631-7
Searching paper 8 of 66 with DOI 10.1007/s11356-020-12143

add all non fetched papers manual

In [14]:
os.makedirs(OUTPUT_DIR, exist_ok=True)

downloaded_dois = set()
new_claims = []

for claim in claims:
    valid_dois = [doi for doi in claim["dois"] if doi]
    if valid_dois:
        new_claims.append(claim)

for claim in new_claims:
    for doi in claim["dois"]:
        if doi not in downloaded_dois:
            try:
                scihub_download(doi, paper_type="doi", out=OUTPUT_DIR)
                downloaded_dois.add(doi)
                print(f"Downloaded: {doi}")
            except Exception as e:
                print(f"Failed to download {doi}: {e}")

# update claims to only include those with successfully downloaded PDFs
filtered_claims = [
    claim for claim in new_claims
    if all(doi in downloaded_dois for doi in claim["dois"])
]

print(f"Original claims: {len(claims)}")
print(f"Filtered claims: {len(filtered_claims)}")
print(f"Total DOIs: {len(downloaded_dois)}")
print(f"All DOIs: {downloaded_dois}")
print(f"Filtered claims: {filtered_claims}")

with open("filtered_claims.json", "w", encoding="utf-8") as f:
    json.dump(filtered_claims, f, indent=4, ensure_ascii=False)

[INFO] | 2024/12/19 23:16:52 | Choose scihub url [0]: https://sci-hub.ru
[INFO] | 2024/12/19 23:16:52 | <- Request: scihub_url=https://sci-hub.ru, source=DoiSource[type=doi, id=10.13106/jafeb.2020.vol7.no12.731], proxies={}


[INFO] | 2024/12/19 23:16:53 | -> Response: status_code=200, content_length=5842
[WARNING] | 2024/12/19 23:16:53 | Error occurs, task status: extracting_failed, error: No pdf tag was found in the given content with the selector: #pdf
[INFO] | 2024/12/19 23:16:53 | Choose scihub url [1]: https://sci-hub.se
[INFO] | 2024/12/19 23:16:53 | <- Request: scihub_url=https://sci-hub.se, source=DoiSource[type=doi, id=10.13106/jafeb.2020.vol7.no12.731], proxies={}
[INFO] | 2024/12/19 23:16:53 | -> Response: status_code=200, content_length=5842
[WARNING] | 2024/12/19 23:16:53 | Error occurs, task status: extracting_failed, error: No pdf tag was found in the given content with the selector: #pdf
[INFO] | 2024/12/19 23:16:53 | Choose scihub url [2]: http://sci-hub.se
[INFO] | 2024/12/19 23:16:53 | <- Request: scihub_url=http://sci-hub.se, source=DoiSource[type=doi, id=10.13106/jafeb.2020.vol7.no12.731], proxies={}
[INFO] | 2024/12/19 23:16:53 | -> Response: status_code=200, content_length=5842
[WARN

Downloaded: 10.13106/jafeb.2020.vol7.no12.731


[INFO] | 2024/12/19 23:16:55 | -> Response: status_code=200, content_length=7974
[INFO] | 2024/12/19 23:16:55 | * Extracted information: {'url': 'https://sci-hub.ru/downloads/2020-12-23/2e/10.3390@su122410514.pdf', 'title': 'Corporate Sustainability  It’s Mine! Effect of Green Product Psychological Ownership on the Environmental Behavior and Performance of Employees. Sustainability, 12(24), 10514'}


100% [==================================================] 603723/603723


...

Downloaded: 10.3390/technologies11020044
Failed to download : 'error'


[INFO] | 2024/12/19 23:20:00 | -> Response: status_code=200, content_length=0
[WARNING] | 2024/12/19 23:20:00 | Error occurs, task status: extracting_failed, error: No pdf tag was found in the given content with the selector: #pdf
[INFO] | 2024/12/19 23:20:00 | Choose scihub url [1]: https://sci-hub.se
[INFO] | 2024/12/19 23:20:00 | <- Request: scihub_url=https://sci-hub.se, source=DoiSource[type=doi, id=10.1109/icitee59582.2023.10317774], proxies={}
[INFO] | 2024/12/19 23:20:00 | -> Response: status_code=200, content_length=0
[WARNING] | 2024/12/19 23:20:00 | Error occurs, task status: extracting_failed, error: No pdf tag was found in the given content with the selector: #pdf
[INFO] | 2024/12/19 23:20:00 | Choose scihub url [2]: http://sci-hub.se
[INFO] | 2024/12/19 23:20:00 | <- Request: scihub_url=http://sci-hub.se, source=DoiSource[type=doi, id=10.1109/icitee59582.2023.10317774], proxies={}
[INFO] | 2024/12/19 23:20:01 | -> Response: status_code=200, content_length=0
[WARNING] | 20

Downloaded: 10.1109/icitee59582.2023.10317774
Failed to download : 'error'
Failed to download : 'error'
Failed to download : 'error'


[INFO] | 2024/12/19 23:20:02 | -> Response: status_code=200, content_length=0
[WARNING] | 2024/12/19 23:20:02 | Error occurs, task status: extracting_failed, error: No pdf tag was found in the given content with the selector: #pdf
[INFO] | 2024/12/19 23:20:02 | Choose scihub url [1]: https://sci-hub.se
[INFO] | 2024/12/19 23:20:02 | <- Request: scihub_url=https://sci-hub.se, source=DoiSource[type=doi, id=10.1111/1748-8583.12524], proxies={}
[INFO] | 2024/12/19 23:20:02 | -> Response: status_code=200, content_length=0
[WARNING] | 2024/12/19 23:20:02 | Error occurs, task status: extracting_failed, error: No pdf tag was found in the given content with the selector: #pdf
[INFO] | 2024/12/19 23:20:02 | Choose scihub url [2]: http://sci-hub.se
[INFO] | 2024/12/19 23:20:02 | <- Request: scihub_url=http://sci-hub.se, source=DoiSource[type=doi, id=10.1111/1748-8583.12524], proxies={}
[INFO] | 2024/12/19 23:20:02 | -> Response: status_code=200, content_length=0
[WARNING] | 2024/12/19 23:20:02 | 

Downloaded: 10.1111/1748-8583.12524
Original claims: 92
Filtered claims: 65
Total DOIs: 66
All DOIs: {'10.1108/SRJ-10-2018-0272', '10.1177/0170840603024003910', '10.48550/arxiv.2305.06472', '10.1177/0007650320930638', 'https://doi.org/10.1002/csr.1480', 'https://doi.org/10/ggkrcg', '10.1177/0007650315584317', '10.5281/zenodo.7644359', '10.3390/su12197999', '10.18559/ebr.2023.2.732', '10.1016/j.jclepro.2019.119114', '10.1109/icitee59582.2023.10317774', '10.1016/j.eiar.2020.106459', '10.1007/s10551-012-1291-2', '10.1111/1748-8583.12524', 'https://doi.org/10.1016/j.jbankfin.2016.12.013', '10.3389/fpsyg.2020.579272', '10.3390/su12219204', '10.1007/s11846-023-00696-z', '10.1111/joms.12514', 'https://doi.org/10.1007/s10551-017-3667-3', '10.1177/000765039703600302', 'https://doi.org/10.1111/ajfs.12244', '10.1007/s12243-023-00980-9', '10.1111/corg.12170', '10.5465/amr.2012.0341', '10.1080/08874417.2023.2261010', '10.3390/s12218987', '10.1002/csr.1480', '10.1016/j.ecolecon.2010.09.029', '10.108

## Categorize the claims 

In [ ]:
from typing import List
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser

class CategoryResponse(BaseModel):
    category: str = Field(description="The category of the given claim")

parser = PydanticOutputParser(pydantic_object=CategoryResponse)

extract_prompt = PromptTemplate(
    input_variables=["claim", "categories"],
    partial_variables={"format_output": parser.get_format_instructions()},
    template="""You have a list of existing categories:
{categories}

Please classify the following claim into one of these categories if it fits well. 
If it does not fit into any of the listed categories, create a new category name.
Here are some examples for Categories you can use, or create new ones if you need to:
"Business", "Sustainability", "Technology", "Economics", "Innovation", etc.

Claim: {claim}

Return only the final chosen or newly created category as a JSON object following the format instructions.

{format_output}
"""
)

extract_chain = extract_prompt | llm | parser

categories = []
claims_w_category = []
for item in filtered_claims:
    categories_str = ", ".join(categories) if categories else "No categories yet."
    response = extract_chain.invoke({"claim": item["claim"], "categories": categories_str})
    chosen_category = response.category.strip()
    if chosen_category not in categories:
        categories.append(chosen_category)

    item['category'] = chosen_category
    claims_w_category.append(item)

print("Final Claims with Categories:", claims_w_category)
print("All Discovered Categories:", categories)
print("Amount of categories:", len(categories))

Final Claims with Categories: [{'claim': 'The accounting sector of a company can promote environmental conservation through environmental costs, and at the same time, improve performance when implementing a Corporate Sustainability Management System (CSMS).', 'references': ['Endiana et al., 2020'], 'dois': ['10.13106/jafeb.2020.vol7.no12.731'], 'doc_source': 's10668-023-02933-7.md', 'id': 0, 'category': 'Sustainability'}, {'claim': 'Allocating appropriate environmental costs through CSMS can effectively improve the company’s financial performance.', 'references': ['Endiana et al., 2020'], 'dois': ['10.13106/jafeb.2020.vol7.no12.731'], 'doc_source': 's10668-023-02933-7.md', 'id': 1, 'category': 'Sustainability'}, {'claim': 'A proper application of CSMS, with the disclosure of environmental activities and costs, can enhance customer loyalty.', 'references': ['Endiana et al., 2020'], 'dois': ['10.13106/jafeb.2020.vol7.no12.731'], 'doc_source': 's10668-023-02933-7.md', 'id': 2, 'category':

In [33]:
from collections import Counter, defaultdict

# 1. Number of Claims in Each Category
category_counts = Counter(item['category'] for item in claims_w_category)
print("\nNumber of Claims in Each Category:")
for category, count in category_counts.items():
    print(f" - {category}: {count}")

# 2. Number of Claims per Paper
claims_per_paper = Counter(item['doc_source'] for item in claims_w_category)
print("\nNumber of Claims per Paper:")
for doc_source, count in claims_per_paper.items():
    print(f" - {doc_source}: {count}")

# 3. Number of Each Category per Paper
categories_per_paper = defaultdict(Counter)
for item in claims_w_category:
    doc_source = item['doc_source']
    category = item['category']
    categories_per_paper[doc_source][category] += 1

print("\nNumber of Each Category per Paper:")
for doc_source, category_counter in categories_per_paper.items():
    print(f"\nPaper: {doc_source}")
    for category, count in category_counter.items():
        print(f"  - {category}: {count}")



Number of Claims in Each Category:
 - Sustainability: 16
 - Corporate Governance and Social Sustainability: 3
 - Corporate Governance: 3
 - Corporate Sustainability: 4
 - Economics: 2
 - Climate Finance: 2
 - Technology: 42
 - Business: 2
 - Financial Technology: 1
 - Innovation: 8
 - Social Sustainability: 1

Number of Claims per Paper:
 - s10668-023-02933-7.md: 19
 - s11301-021-00211-2 (1).md: 5
 - ssrn-3708495 (2).md: 5
 - Use_Cases_for_Generative_AI_in_organizations.md: 55

Number of Each Category per Paper:

Paper: s10668-023-02933-7.md
  - Sustainability: 14
  - Corporate Governance and Social Sustainability: 3
  - Corporate Governance: 1
  - Corporate Sustainability: 1

Paper: s11301-021-00211-2 (1).md
  - Corporate Governance: 2
  - Corporate Sustainability: 2
  - Sustainability: 1

Paper: ssrn-3708495 (2).md
  - Economics: 1
  - Corporate Sustainability: 1
  - Climate Finance: 2
  - Sustainability: 1

Paper: Use_Cases_for_Generative_AI_in_organizations.md
  - Technology: 42
 

## Bundle and Merge Claims

In [ ]:
from collections import defaultdict
from langchain.prompts import PromptTemplate

for idx, claim in enumerate(claims_w_category):
    claim["id"] = idx

claims_by_category = defaultdict(list)
for c in claims_w_category:
    claims_by_category[c["category"]].append(c)

all_claims = [c for cat_list in claims_by_category.values() for c in cat_list]

merge_prompt = PromptTemplate(
    input_variables=["claims_text"],
    template="""You are a writing assistant.
Merge the following claims into a single cohesive claim that captures all their key points but don't merely summarize them. Integrate all elements so that the unified claim represents them collectively:

Claims:
{claims_text}

Return one paragraph unifying these claims.
"""
)

group_size = 3
merged_claims = []

def get_other_category_claim(current_category, used_claims):
    for c in all_claims:
        if c["category"] != current_category and c["id"] not in used_claims:
            return c
    return None

def get_other_document_claim(current_doc_source, used_claims):
    for c in all_claims:
        if c["doc_source"] != current_doc_source and c["id"] not in used_claims:
            return c
    return None

used_claims = set()

for cat, cat_claims in claims_by_category.items():
    for i in range(0, len(cat_claims), group_size):
        group = cat_claims[i:i+group_size]

        # if not enough claims in this category, try adding another category claim
        if len(group) < group_size:
            other_claim = get_other_category_claim(cat, used_claims)
            if other_claim is not None:
                group.append(other_claim)
                used_claims.add(other_claim["id"])

        if len(group) < 2:
            continue

        # check category diversity
        group_categories = {g["category"] for g in group}
        if len(group_categories) == 1:
            other_claim = get_other_category_claim(cat, used_claims)
            if other_claim is not None:
                replaced = group.pop()
                group.append(other_claim)
                used_claims.add(other_claim["id"])

        # check document diversity
        group_doc_sources = {g["doc_source"] for g in group}
        if len(group_doc_sources) == 1:
            current_doc_source = group[0]["doc_source"]
            other_doc_claim = get_other_document_claim(current_doc_source, used_claims)
            if other_doc_claim is not None:
                replaced = group.pop()
                group.append(other_doc_claim)
                used_claims.add(other_doc_claim["id"])

        # mark all claims in this group as used
        for g in group:
            used_claims.add(g["id"])

        claims_text = "\n".join([f"- {g['claim']}" for g in group])
        merged_claim = (merge_prompt | llm).invoke({"claims_text": claims_text})

        all_refs = []
        all_cats = []
        all_doc_sources = []
        for g in group:
            all_refs.extend(g.get("references", []))
            all_cats.append(g["category"])
            all_doc_sources.append(g["doc_source"])

        all_refs = list(set(all_refs))
        all_cats = list(set(all_cats))
        all_doc_sources = list(set(all_doc_sources))

        merged_claims.append({
            "original_claims": group,
            "merged_claim": merged_claim.content.strip(),
            "categories": all_cats, 
            "merged_references": all_refs, 
            "doc_sources": all_doc_sources 
        })

print("Merged Claims:")
for mc in merged_claims:
    print(mc)


Merged Claims:
{'original_claims': [{'claim': 'The accounting sector of a company can promote environmental conservation through environmental costs, and at the same time, improve performance when implementing a Corporate Sustainability Management System (CSMS).', 'references': ['Endiana et al., 2020'], 'dois': ['10.13106/jafeb.2020.vol7.no12.731'], 'doc_source': 's10668-023-02933-7.md', 'id': 0, 'category': 'Sustainability'}, {'claim': 'Allocating appropriate environmental costs through CSMS can effectively improve the company’s financial performance.', 'references': ['Endiana et al., 2020'], 'dois': ['10.13106/jafeb.2020.vol7.no12.731'], 'doc_source': 's10668-023-02933-7.md', 'id': 1, 'category': 'Sustainability'}, {'claim': 'Environmental performance leads to better financial performance.', 'references': ['Hang et al., 2019', 'Endrikat, 2016', 'Endrikat et al., 2014', 'Albertini, 2013', 'Dixon-Fowler et al., 2013'], 'dois': ['10.1002/bse.2172', '10.1007/s10551-015-2631-7', '10.1016/

In [34]:

question_prompt = PromptTemplate(
    input_variables=["merged_claim"],
    template=
    """You have the following merged claim:
    {merged_claim}
    
    Generate a single, question that, if answered, would be naturally resolved by this claim.
    This question should:
    - Integrate insights from multiple logically connected business domains (e.g., finance, sustainability, technology, operations).
    - Require synthesizing and validating information potentially from multiple sections or sources, reflecting the complexity of cross-document or intra-document retrieval.
    - Demand abstract reasoning and strategic-level interpretation, going beyond simple fact retrieval.
    
    The resulting question should be global in scope, and should not be answerable without the information provided in the claim.
    """
)

final_dataset = []

for mc in merged_claims:
    question_response = (question_prompt | llm).invoke({"merged_claim": mc["merged_claim"]})
    question = question_response.content.strip()
    
    entry = {
        "question": question,
        "ground_truth": mc["merged_claim"],
        "references": mc["merged_references"],
        "categories": mc["categories"],
        "doc_sources": mc["doc_sources"]
    }
    
    final_dataset.append(entry)


with open(DATASET, "w", encoding="utf-8") as f:
    json.dump({"responses": final_dataset}, f, indent=4, ensure_ascii=False)


for entry in final_dataset:
    print(json.dumps(entry, indent=4, ensure_ascii=False))

{
    "question": "How can integrating environmental costs into a Corporate Sustainability Management System (CSMS) create a synergistic effect that aligns sustainability with profitability, enhancing both environmental conservation and financial performance across various business domains such as finance, operations, and technology?",
    "ground_truth": "Integrating environmental costs into a company's accounting sector through a Corporate Sustainability Management System (CSMS) not only fosters environmental conservation but also enhances overall performance, demonstrating a direct link between environmental and financial success. By strategically allocating environmental costs within a CSMS, companies can effectively boost their financial outcomes, illustrating that improved environmental performance inherently leads to superior financial performance. Thus, adopting a comprehensive CSMS aligns environmental objectives with financial goals, creating a synergistic effect that propels

In [35]:
print(len(final_dataset))

32
